In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import numpy as np
import pandana as pdna
import geopandas as gpd
import pandas as pd
import math
import networkx as nx
import sys
# adding functions 
sys.path.insert(0, 'C:\\Users\\z3258367\\OneDrive - UNSW\\#PhD\\Walkability\\Other Cities\\Open-Walk-Index')
from walkability_functions import *

Choose a projected CRS to be used for all distance calculations.

In [3]:
proj_crs = "EPSG:7856"

## Import Data

Data sources:
1. Shape of Greater Sydney - used to clip points
2. Points of interest from OSM
3. Transport for NSW public transport stops
4. Spatial Services NSW for additional POIs
5. Employment data - processed from ABS originally

In [4]:
folder = "C:\\Users\\z3258367\\OneDrive - UNSW\\#PhD\\"
Greater_Sydney = gpd.read_file((folder + 
    "Data\\greater_sydney_shape\\Greater_Sydney_Dissolve.shp")
    ).to_crs(proj_crs)

In [5]:
osm_poi_points = gpd.read_file(''.join((folder, 
    "Walkability\\Mavoa\\Pandana inputs\\Greater_Sydney_OSM_POIs_sp.shp")))
osm_poi_areas = gpd.read_file(folder + 
    "Walkability\\Mavoa\\Pandana inputs\\Greater_Sydney_OSM_POIs_a_sp.gpkg")
osm_transport_points = gpd.read_file(folder +
    "Walkability\\Mavoa\\Pandana inputs\\Greater_Sydney_OSM_transport_sp.shp")
osm_transport_areas =  gpd.read_file(folder +
    "Data\\OSM-australia-latest-free\\gis_osm_transport_a_free_1.shp")

In [6]:
tfnsw = gpd.read_file(folder +
                    "Data\\TfNSW\\full_greater_sydney_gtfs_static\\unique_stops_types.csv")
tfnsw.geometry = gpd.points_from_xy(tfnsw['stop_lon'], tfnsw['stop_lat'])
tfnsw.crs = "EPSG:7843"
tfnsw = tfnsw.to_crs(proj_crs)

In [7]:
# import SS NSW data
SS_NSW = gpd.read_file("C:\\Users\\z3258367\\OneDrive - UNSW\\#PhD\\Data\\NSW Spatial Services\\NSW_Features_of_Interest_Category.gdb",layer='BuildingComplexPoint')
SS_NSW.to_crs(Greater_Sydney.crs, inplace = True)
SS_NSW['fclass'] = (SS_NSW['classsubtype'].astype(str) + "-" 
                    + SS_NSW['buildingcomplextype'].astype(str))

# add general cultural points
SS_NSW_gc = gpd.read_file("C:\\Users\\z3258367\\OneDrive - UNSW\\#PhD\\Data\\NSW Spatial Services\\NSW_Features_of_Interest_Category.gdb",layer='GeneralCulturalPoint')
SS_NSW_gc.to_crs(Greater_Sydney.crs, inplace = True)
SS_NSW_gc['fclass'] = (SS_NSW_gc['classsubtype'].astype(str) + "-" 
                       + SS_NSW_gc['generalculturaltype'].astype(str) + "-gc")

SS_NSW = pd.concat([SS_NSW, SS_NSW_gc]).to_crs(proj_crs)

In [8]:
employment_centrs = gpd.read_file(folder +
    "Data\\Colouring\\Centroids employment MBs.gpkg").to_crs(proj_crs)

Convert polygonal datasets to points and any multipart datasets to single part.

In [9]:
osm_pois_2 = single_points(osm_poi_areas)
osm_transport_2 = single_points(osm_transport_areas)

osm_df = pd.concat([osm_poi_points, osm_pois_2, osm_transport_points, 
                    osm_transport_2]).to_crs(proj_crs)

### Categorise and weight POIs

Categorise POI data - change classes depending on your analysis and your data sources.

In [11]:
SS_names = {'shopping centre':["4-10"], 'post office':["2-18"],'place of worship':["2-16"], 'health centre':["3-3", "3-5", "3-6"], 
                 'school':["1-1", "1-2", "1-3", "1-5", "1-6", "1-8"], 'university':["1-4"], 'childcare':["1-7"],
                 'art gallery':["2-2"], 'library': ["2-11"], 'museum':["2-14"], 'sports centre':["6-12"], 
                 'zoo':["6-18"], 'outdoor theater':["1-6-gc"], 'swimming pool':["6-15", "9-2-gc"], 
                 'tourist attraction':["6-17"], 'golf course':["1-2-gc"], 'lookout':["1-5-gc"], 
                 'park':["1-7-gc"], 'picnic area':["1-8-gc"], 'sports field':["1-12-gc", "1-13-gc"]}
SS_NSW = categorise_pois(SS_NSW, SS_names, old_column='fclass', new_column='fclass')

Tags present in the dataset but not categorised:
['4-6' '2-24' '5-9' '2-0' '5-10' '2-8' '2-12' '3-0' '2-4' '6-3' '2-17'
 '2-21' '2-20' '2-9' '5-8' '4-7' '4-4' '4-8' '2-23' '1-0' '3-1' '2-19'
 '2-5' '4-0' '2-6' '2-7' '2-1' '5-1' '4-1' '2-10' '2-15' '5-5' '5-11'
 '5-2' '4-2' '4-3' '5-3' '4-9' '5-6' '3-4' '5-7' '6-0' '5-0' '6-2'
 '4-11-gc' '12-0-gc' '8-0-gc' '4-12-gc' '1-4-gc' '4-7-gc' '7-0-gc'
 '4-6-gc' '4-2-gc' '8-2-gc' '6-5-gc' '1-14-gc' '12-1-gc' '1-1-gc' '7-2-gc'
 '7-1-gc' '1-11-gc' '4-9-gc' '4-3-gc' '5-0-gc' '5-1-gc' '4-0-gc' '4-4-gc'
 '6-2-gc' '2-0-gc' '4-8-gc' '2-1-gc' '8-1-gc' '1-3-gc' '3-1-gc' '6-4-gc'
 '7-3-gc' '4-5-gc' '2-2-gc' '6-7-gc' '7-4-gc' '6-8-gc' '6-6-gc' '6-3-gc'
 '2-3-gc' '1-10-gc' '4-13-gc' '1-0-gc' '7-5-gc' '9-0-gc' '9-1-gc' '6-1-gc'
 '3-0-gc' '6-0-gc']


In [12]:
tfnsw_categories = {'transport':['bus'], 'trains':['train','ferry']}

tfnsw_categorised = categorise_pois(tfnsw, tfnsw_categories, 
                                 old_column='service_type', new_column='category')

SS_categories = {'shopping':["4-10"], 
                 'errands':["2-18", "2-16", "3-3", "3-5", "3-6"], 
                 'education':["1-1", "1-2", "1-3", "1-4", "1-8"], 
                 'entertainment':["2-2", "2-11", "2-14", "6-12", "6-18", "1-6-gc"], 
                 'parks':["6-15", "6-17", "1-2-gc", "1-5-gc", "1-7-gc", 
                           "1-8-gc", "1-12-gc", "1-13-gc", "9-2-gc"]}

SS_categorised = categorise_pois(SS_NSW, SS_categories, 
                                 old_column='fclass', new_column='category')

osm_categories = {"eating" : ['restaurant', 'pub', 'cafe', 'fast_food', 
                              'food_court', 'bakery', 'bar', 'nightclub', 'biergarten'], 
                  'groceries' : ['supermarket', 'chemist', 'pharmacy', 'greengrocer', 
                                 'convenience', 'butcher', 'beverages', 'alcohol'], 
                  'shopping' : ['mall', 'bicycle_shop', 'clothes', 
                                'department_store', 'doityourself', 
                                'outdoor_shop', 'stationery', 'bookshop', 
                                'gift_shop', 'newsagent', 'car_dealership', 
                                'kiosk', 'furniture_shop', 'sports_shop', 
                                'garden_centre', 'computer_shop', 'shoe_shop', 
                                'beauty_shop', 'florist', 'video_shop', 'toy_shop', 
                                'mobile_phone_shop', 'jeweller', 'travel_agent'], 
                  'errands' : ['post_box', 'post_office', 'bank', 'atm',
                               'doctors', 'dentist', 'laundry', 'hospital',
                               'car_wash', 'veterinary', 'hairdresser', 'optician'], 
                  'parks' : ['viewpoint', 'park', 'playground', 'picnic_site', 
                             'pitch', 'swimming_pool', 'sports_centre', 
                             'golf_course', 'track', 'dog_park'], 
                  'education' : ['college', 'school', 'kindergarten', 'university'], 
                  'entertainment' : ['library', 'attraction', 'stadium', 
                                     'arts_centre', 'theatre', 'artwork', 
                                     'archaeological', 'cinema', 'museum', 
                                     'ruins', 'observation_tower', 
                                     'community_centre', 'zoo', 'castle', 
                                     'theme_park', 'ice_rink'], 
                 'trains' : ['ferry_terminal', 'railway_station', 'bus_station', 
                             'tram_stop', 'railway_halt', 'publictransport'], 
                 'transport' : ['car_sharing', 'bus_stop']}

osm_categorised = categorise_pois(osm_df, osm_categories, 
                                  old_column='fclass', new_column='category')

Tags present in the dataset but not categorised:
['nan']
Tags present in the dataset but not categorised:
['place of worship' 'tourist attraction' 'swimming pool' 'school'
 'shopping centre' 'post office' 'sports centre' 'library' 'university'
 'museum' 'health centre' 'zoo' 'art gallery' 'childcare' 'lookout'
 'picnic area' 'park' 'sports field' 'golf course' 'outdoor theater']
Tags present in the dataset but not categorised:
['drinking_water' 'toilet' 'memorial' 'public_building' 'monument'
 'police' 'motel' 'shelter' 'tourist_info' 'bicycle_rental' 'hostel'
 'hotel' 'telephone' 'recycling' 'guesthouse' 'lighthouse' 'courthouse'
 'fire_station' 'camp_site' 'town_hall' 'fountain' 'caravan_site' 'bench'
 'water_tower' 'comms_tower' 'tower' 'waste_basket' 'graveyard'
 'wastewater_plant' 'camera_surveillance' 'nursing_home' 'car_rental'
 'recycling_clothes' 'vending_parking' 'battlefield' 'chalet'
 'vending_any' 'prison' 'vending_machine' 'water_works' 'embassy'
 'water_well' 'windmill' 

Need to remove potential overlap between different data sources (and inside some data sources). For this dataset it's around 30% because there is overlap of public transport stops between OSM and TfNSW, and overlap of things like post offices between OSM and SSNSW. Then take this combined POI set and clip it to the study area: should be the same area as is covered by the network. This is important otherwise points outside the network may be erroneously linked to the network.

In [13]:
pois = remove_duplicate_pois([osm_categorised, SS_categorised, 
                              tfnsw_categorised], buffer=10)

pois = gpd.clip(pois, Greater_Sydney)

Removed 8.68% duplicate points from dataframes


Choose walk index weightings, and output the sums of each category and the total to check. The walk index will be out of 100 regardless of this sum, but it is important to note that eg. shopping is only '10% of the walk index' if shopping is 10 out of 100.

In [14]:
poi_weights = {
    "employment": [10],
    "eating": [3, 3, 3, 2, 2, 1, 1, 1, 1, 1],
    "groceries": [10, 4],
    "shopping": [2, 2, 2, 2, 2],
    "errands": [6, 2, 4],
    "parks": [6],
    "education": [10],
    "entertainment": [5],
    "trains": [10],
    "transport": [2.5, 2.5]
}

In [15]:
category_sums = {k: sum(v) for k, v in poi_weights.items()}
total = sum(category_sums.values())
print(category_sums)
print("total: ", total)

{'employment': 10, 'eating': 18, 'groceries': 14, 'shopping': 10, 'errands': 12, 'parks': 6, 'education': 10, 'entertainment': 5, 'trains': 10, 'transport': 5.0}
total:  100.0


### Import network

In this case the network is already in the same projected CRS as everything else but I have left in the transformation to be clear.

In [17]:
# reading directly with geopandas.read_file crashes on my computer so I read into pandas then convert to gdf instead
data = "C:\\Users\\z3258367\\OneDrive - UNSW\\#PhD\\Walkability\\Other Cities\\Colouring data & results\\Sydney Data\\"
edges_df = pd.read_csv(data + "Data\\colouring_edges_150322.csv")
nodes_df = pd.read_csv(data + "Data\\colouring_nodes_150322.csv")
edges = gpd.GeoDataFrame(edges_df, 
                         geometry=gpd.GeoSeries.from_wkt(edges_df['geometry'])).set_crs(proj_crs)
nodes = gpd.GeoDataFrame(nodes_df, 
                         geometry=gpd.GeoSeries.from_wkt(nodes_df['geometry'])).set_crs(proj_crs)
edges = edges.to_crs(proj_crs)
nodes = nodes.to_crs(proj_crs)

C:\Users\z3258367\Anaconda3\envs\ox\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (2,4,5,7,9,13,14,19,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\z3258367\Anaconda3\envs\ox\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (6,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'str'>
<class 'str'>


Pandana expects edges to have a two item index based on the same IDs as the node index.

In [18]:
nodes.set_index('connect_id',inplace=True)

edges['from_idx'] = edges['from']
edges['to_idx'] = edges['to']
edges= edges.set_index(['from_idx', 'to_idx'])
edges.index.names= ['from_idx','to_idx']

Pandana network creation.

In [19]:
distance_network = pdna.Network(nodes['x'], nodes['y'],
                                   edges['from'], edges['to'], 
                                   edges[['length']])

maximum_dist = 2400

Pandana network querying. The 'employment' category is empty because we didn't add the employment points to the POI dataset.

In [34]:
results = walk_index_m(distance_network, pois, poi_weights, distance=maximum_dist)  

Category employment is empty
Finished category: eating
Finished category: groceries
Finished category: shopping
Finished category: errands
Finished category: parks
Finished category: education
Finished category: entertainment
Finished category: trains
Finished category: transport


### Employment

The current approach is to find up to 100 closest employment nodes within the maximum distance. Then look up the number of jobs at each one, apply a distance decay function to each distance, multiply these together, and sum.

An alternative approach which would be more convenient would be to use the Pandana 'aggregate' function which aggregates from all nodes within the maximum distance. However, there is limited ability to change the distance decay rate within the aggregation function. It can either be flat (no decay), linear (going to 0 at the max distance), or exponential where beta is set as 1/max distance. For walking I would like a beta of 0.001, but this requires the radius to be 1000m. If the radius is 2400m, beta is only 0.0004. This can be changed in the future if the Pandana function is updated to take a decay parameter.

In [35]:
x, y = (employment_centrs['geometry'].x, employment_centrs['geometry'].y)

distance_network.set_pois(category='employment', maxdist=maximum_dist, maxitems=100, x_col=x, y_col=y)

employment_access = distance_network.nearest_pois(
    distance=maximum_dist, category='employment', num_pois=100, include_poi_ids=True)

The nearest_pois function returns both distances and the IDs of the nearest pois (with include_poi_ids option). The IDs can then be used to retrieve the number of jobs at each point. I found a merge was the fastest way to join this data.

In [36]:
def itermerge(dataframe, jobs):
    i=0
    for column in dataframe:
        dataframe = dataframe.merge(jobs, how='left', left_on = column, right_index = True, suffixes = [None, i])
        i = i + 1
    return dataframe

In [37]:
jobcounts = itermerge(employment_access.iloc[:,100:200], employment_centrs['Jobs_count'])

results['jobs'] = ((employment_access.iloc[:,0:100].applymap(access_weight, distance=maximum_dist))*
                                jobcounts.iloc[:,100:200].values
                                ).sum(axis=1)

weight = 100*sum(poi_weights['employment'])/sum(sum(list(poi_weights.values()),[]))

results['employment'] = weight*results['jobs']/max(results['jobs'])

results['Walk_Index'] = results['Walk_Index'] + results['employment']

## Export results

Filter the results to the original Colouring Sydney buildings only. Optionally export results as a csv.

In [38]:
building_results = results.filter(items=nodes[nodes['connect_type'] == 'poi'].index, axis=0)

In [39]:
building_results.to_csv("Colouring_results_250321.csv")

Import building footprints and join the data to them, then export these polygons.

In [40]:
results_gdf = gpd.GeoDataFrame(building_results, geometry = gpd.GeoSeries.from_xy(building_results.x, building_results.y, crs="EPSG:7856"))

buildings_foot = gpd.read_file(folder +
    "Data\\Colouring\\Building Footprints\\sydney_bf.shp").to_crs(proj_crs)

# join to data
buildings_foot = gpd.sjoin(buildings_foot, results_gdf, how='left', predicate='contains')

buildings_foot.to_file("Colouring_bf_results_250321.gpkg")